<a href="https://colab.research.google.com/github/amara929/amara929/blob/main/Dask_Vivian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dask[complete] dask-ml --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 9.7 MB/s eta 0:00:00


In [ ]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client

# Initialize Dask Client (Single Machine Mode)
client = Client()
print(client)

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:38047
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42215'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39409'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:41205 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:41205
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:56160
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:46345 name: 1
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:46345
INFO:distributed.core:Starting established connection to tcp://127

<Client: 'tcp://127.0.0.1:38047' processes=2 threads=2, memory=12.67 GiB>


In [ ]:
import pandas as pd
import numpy as np

# Create large pandas Daraframe (Simulating Big Data)
n_rows = 1_000_000 # 1 million rows
df = pd.DataFrame({
    "id": np.arange(n_rows),
    "value": np.random.randint(1, 100, n_rows),
    "category": np.random.choice(["A", "B", "C"], n_rows)
})

# Convert Pandas Dataframe to Dask DataFrame
ddf = dd.from_pandas(df, npartitions=10) #Split into 10 partitions

In [ ]:
# Display the first few rows (lazy execution)
print(ddf.head())

# Get data types (metadata only, doesn't load full data)
print(ddf.dtypes)

   id  value category
0   0     29        C
1   1     31        C
2   2     10        B
3   3     64        C
4   4     14        A
id                    int64
value                 int64
category    string[pyarrow]
dtype: object


In [ ]:
# Display the first few rows (lazy execution)
print(ddf.head())

# Get data types (metadata only, doesn't load full data)
print(ddf.dtypes)

# Perform filtering (lazy execution)
filtered_ddf = ddf[ddf["value"] > 50]

# Perform aggregations (Lazy Execution)
grouped_ddf = ddf.groupby("category")["value"].mean()

# Compute results (Trigger execution)
print(filtered_ddf.compute().head()) # Convert to pandafor viewing
print(grouped_ddf.compute()) # Perform actual aggregation

   id  value category
0   0     29        C
1   1     31        C
2   2     10        B
3   3     64        C
4   4     14        A
id                    int64
value                 int64
category    string[pyarrow]
dtype: object


/usr/local/lib/python3.11/dist-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.85 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


   id  value category
3   3     64        C
5   5     97        C
6   6     89        B
7   7     73        C
8   8     69        A


/usr/local/lib/python3.11/dist-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 16.22 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


category
A    50.013463
B    49.984288
C    50.019296
Name: value, dtype: float64


In [ ]:
# Create a large Dask Array (10 million elements, chunked)
arr = da.random.random(size=(10_000_000,), chunks=(1_000_000,))

# Compute mean and sum (Lazy untill '.compute()' is called)
mean_value = arr.mean().compute()
sum_value = arr.sum().compute()

print("Mean:", mean_value)
print("Sum:", sum_value)

Mean: 0.5001125785718387
Sum: 5001125.785718387


In [ ]:
# Create a large Dask Array (10 million elements, chunked)
arr = da.random.random(size=(10_000_000,), chunks=(1_000_000,))

# Compute mean and sum (Lazy untill '.compute()' is called)
mean_value = arr.mean().compute()
sum_value = arr.sum().compute()

print("Mean:", mean_value)
print("Sum:", sum_value)

Mean: 0.499932828945583
Sum: 4999328.28945583


In [ ]:
from dask_ml.linear_model import LinearRegression

# Create a synthetic dataset
x = da.random.random((10_000, 2), chunks=(1_000, 2))
y = da.random.random(10_000, chunks=(1_000,))

# Train a scalable Linear Regression Model
model = LinearRegression()
model.fit(x, y)

# Predict values
predictions = model.predict(x)
print(predictions.compute()[:5])

[0.49225815 0.49767491 0.4885859  0.48035649 0.49980919]


In [ ]:
client.close()

INFO:distributed.scheduler:Remove client Client-a28e8d5f-0ae2-11f0-80f8-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:33808; closing.
INFO:distributed.scheduler:Remove client Client-a28e8d5f-0ae2-11f0-80f8-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-a28e8d5f-0ae2-11f0-80f8-0242ac1c000c
INFO:distributed.scheduler:Retire worker addresses (stimulus_id='retire-workers-1743063907.8471859') (0, 1)
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:42349'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:44543'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:57062; closing.
INFO:distributed.scheduler:Remove worker addr: tcp://127.0.0.1:39415 name: 0 (stimulus_id='handle-worker-cleanup-1743063907.8688807')
INFO:distr